In [3]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from urllib.request import urlopen
from json import load
sns.set_style('white')
sns.set_context('notebook')

# 1. Import Data

In [4]:
# ASC Cohort
conn = sqlite3.connect('./Data/knwlg_blf.db')
#dat1 = pd.read_sql("SELECT * FROM trials A INNER JOIN subjects B ON A.prolific_id = B.prolific_id WHERE B.completion_code LIKE '%XXX%'", con=conn)
dat1 = pd.read_sql("SELECT * FROM trials A INNER JOIN subjects B ON A.prolific_id = B.prolific_id "
                   "WHERE B.block1_complete=TRUE "
                   "AND B.block2_complete=TRUE "
                   "AND B.block3_complete=TRUE "
                   "AND B.completion_code LIKE '%XXX%'", con=conn)
dat1 = dat1.loc[:,~dat1.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate
# REMOVE DISTRACTORS
dat1 = dat1.loc[dat1.trial_type == 'test'].reset_index(drop=True)
dat1['group'] = 'Autism'

# TD cohort
conn2 = sqlite3.connect('./Data/knwlg_blf_controls.db')
dat2 = pd.read_sql('SELECT * FROM trials A INNER JOIN subjects B ON A.prolific_id = B.prolific_id WHERE B.completion_code="548DA3BD"', con=conn2)
dat2 = dat2.loc[:,~dat2.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate
# REMOVE DISTRACTORS
dat2 = dat2.loc[dat2.trial_type == 'test'].reset_index(drop=True)
dat2['email'] = 'N/A'
dat2['group'] = 'Neurotypical'

# Combine into one big dataframe
dat = pd.concat(objs=[dat1,dat2], ignore_index=True)
#
dat['email'] = dat.email.str.lower().str.strip()

# data saved for testing purposes has id as float instead of alphanumeric so only keep alphanumeric IDs
dat = dat[dat.prolific_id.str.isalnum()].reset_index(drop=True)
dat['response_key'] = dat.response_key.apply(literal_eval)

## Cleaning
### Get Email addressed for reimbursements

In [ ]:
emails = dat # pd.read_sql('SELECT * FROM demographics A INNER JOIN subjects B ON A.prolific_id = B.prolific_id WHERE B.completion_code="XXXX"', con=conn)

emails = emails.loc[(emails.block1_complete == True) & (emails.block2_complete == True) & (emails.block3_complete == True)]#.email.to_csv('emails.csv', sep=',')
emails = emails.loc[:,~emails.columns.duplicated()].copy()
#emails['emailed'] = '8-24-2022'
#emails['paid']= '8-24-2022'
emails = emails[['prolific_id', 'participation_date', 'email']]
emails['email'] = emails.email.str.strip()
emails.email.sort_values().unique()
#ed.to_csv('Data/emails2.csv', sep=',')

### Note where subjects entered invalid responses

In [5]:
accepted_answers = ['f', 'j', '999']
dat['accepted_answer'] = None
for i, response in enumerate(dat.response_key):
    if sum([key.lower() in accepted_answers for key in response]) >= 1:  # == len(response):
        dat.loc[i, 'accepted_answer'] = True
    else:
        dat.loc[i, 'accepted_answer'] = False

print(f"{len(dat.loc[dat.accepted_answer == False].prolific_id.unique())} participants responded at least one trial with invalid keys. \n"
      f"{len(dat.loc[(dat.accepted_answer == False)&(dat.group == 'Neurotypical')].prolific_id.unique())} of these are from the neurotypical group. \n"
      f" {len(dat.loc[(dat.accepted_answer == False)&(dat.group == 'Autism')].prolific_id.unique())} of these are from the autism group.")

26 participants responded at least one trial with invalid keys. 
16 of these are from the neurotypical group. 
 10 of these are from the autism group.


### Convert SQL strings to python datetime object and compute timedeltas

In [21]:
dat['response_onset'] = dat.apply(lambda row: datetime.datetime.strptime(row['response_onset'], '%Y-%m-%d %H:%M:%S.%f'), axis=1)
dat['target_onset'] = dat.apply(lambda row: datetime.datetime.strptime(row['target_onset'], '%Y-%m-%d %H:%M:%S.%f'), axis=1)
dat['participation_date'] = dat.apply(lambda row: datetime.datetime.strptime(row['participation_date'], '%Y-%m-%d %H:%M:%S.%f'), axis=1)
dat['rt'] = dat.apply(lambda row: row['response_onset'] - row['target_onset'], axis=1)
dat['rt_ms'] = dat.apply(lambda row: int(round(row['rt'].total_seconds() * 1e3)), axis=1)

# 2. Accuracy

### Note trials where subjects timed-out after the 5-seconds

In [22]:
dat['timeout'] = False
# Here are timeouts without a response at all
dat.loc[dat.apply(lambda row: '999' in row['response_key'], axis=1), 'timeout'] = True
# Here is where pps pressed a key after 5 seconds, but before the experiment redirected
timeouts = dat.loc[(dat.response_key.apply(len)>1) & (dat.apply(lambda row: '999' in row['response_key'], axis=1))].index
dat.loc[timeouts, 'timeout'] = True

print(f"{len(dat.loc[dat.timeout == True].prolific_id.unique())} participants timed out during at least one trial. \n"
      f"{len(dat.loc[(dat.timeout == True)&(dat.group == 'Neurotypical')].prolific_id.unique())} of these are from the neurotypical group. \n"
      f" {len(dat.loc[(dat.timeout == True)&(dat.group == 'Autism')].prolific_id.unique())} of these are from the autism group.")

277 participants timed out during at least one trial. 
176 of these are from the neurotypical group. 
 101 of these are from the autism group.


In [23]:
# If subjects hit more than one key, use only the last one
dat['response_key'] = [resp[-1].lower() for resp in dat.response_key]

In [24]:
#  correct answers
dat.loc[dat.belief_type == 'TB', 'correct_answer'] = 'j'
dat.loc[dat.belief_type != 'TB', 'correct_answer'] = 'f'
# update correct column based on new answer key
dat['correct'] = dat.apply(lambda row: row['response_key'] == row['correct_answer'], axis=1)

# timeouts are considered incorrect
dat.loc[dat.timeout == True, 'correct'] = False

## Time to Complete

In [37]:
#dat.participation_date
dat.completion_time - dat.participation_date

TypeError: unsupported operand type(s) for -: 'str' and 'str'

## Exclude participants with mean accuracy < .6

In [ ]:
dat.loc[dat.group == 'Autism'].groupby('prolific_id').correct.mean().hist()

In [ ]:
print(f'N autism cohort before exclusion: {len(dat.loc[dat.group == "Autism"].prolific_id.unique())}')
## Exclude participants with mean accuracy < .6
xx = dat.groupby('prolific_id').correct.mean() > .6
dat = dat.loc[dat.prolific_id.isin(xx[xx == True].index.tolist())]

In [ ]:
print(f'N autism cohort after exclusion: {len(dat.loc[dat.group == "Autism"].prolific_id.unique())}')

In [ ]:
v_dat = dat#.loc[(dat.accepted_answer == True)]
acc = pd.DataFrame(columns=['cohort', 'agent_state', 'ascription', 'pct_correct', 'pct_incorrect'])
for gro in dat['group'].unique():
    for bel in dat.belief_type.unique():
        for ascrip in dat.ascription_type.unique():
            dd = v_dat.loc[(v_dat.group == gro)&(v_dat.belief_type==bel)&(v_dat.ascription_type==ascrip)]
            pc = len(dd.loc[dd.correct== True])/len(dd)
            ndf = pd.DataFrame({'cohort':[gro], 'agent_state':[bel], 'ascription':[ascrip], 'pct_correct':[pc], 'pct_incorrect':[1-pc]})
            acc = pd.concat([acc,ndf ], axis=0)#.reset_index(drop=True)
#sns.barplot(x='belief_type', row='group', x='ascription_type', hue='correct', kind='bar', data=)
#sns.histplot(data=v_dat, x='belief_type', hue='correct', multiple='stack', element='bars', stat='percent')

In [ ]:
import matplotlib.patches as mpatches
acc['total'] = 1
acc.melt(id_vars=['cohort', 'agent_state', 'ascription'], value_vars=['pct_correct', 'total'], value_name='Percent', var_name='Accuracy')
#sns.catplot(col='belief_type', row='group', x='correct', hue='ascription_type', kind='count', data=dat)
#dat.groupby('group').prolific_id.describe()
# bar chart 1 -> top bars (group of 'smoker=No')
#f, ax = plt.subplots(1, 2)
#bar1 = sns.barplot(x="agent_state",  y="total", hue='ascription', data=acc.loc[acc.cohort == 'Neurotypical'], color='darkblue', ax=ax[0])
#bar1a = sns.barplot(x="agent_state",  y="total", hue='ascription', data=acc.loc[acc.cohort == 'Autism'],  color='darkblue', ax=ax[1])

# bar chart 2 -> bottom bars (group of 'smoker=Yes')
#bar2 = sns.barplot(x="agent_state",  y="pct_correct", hue='ascription', data=acc.loc[acc.cohort == 'Neurotypical'], color='darkblue', ax=ax[0])
#bar2a = sns.barplot(x="agent_state",  y="pct_correct", hue='ascription', data=acc.loc[acc.cohort == 'Autism'], color='darkblue', ax=ax[1])

bar2 = sns.catplot(x="agent_state", y="pct_correct", hue='ascription', data=acc,col='cohort', kind='bar')
# add legend


In [ ]:
total = v_dat.groupby(['belief_type', 'ascription_type'])['accepted_answer'].sum().reset_index()
correct = v_dat[v_dat.correct == True].groupby(['belief_type', 'ascription_type'])['accepted_answer'].sum().reset_index()

correct['pct'] = [i / j * 100 for i,j in zip(len(correct), len(total))]
total['pct'] = [i / j * 100 for i,j in zip(len(total), len(total))]

# bar chart 1 -> top bars (group of 'smoker=No')
bar1 = sns.barplot(x="belief_type",  y="pct", data=total, color='darkblue')

# bar chart 2 -> bottom bars (group of 'smoker=Yes')
bar2 = sns.barplot(x="belief_type", y="pct", data=correct, color='lightblue')

# add legend
top_bar = mpatches.Patch(color='darkblue', label='incorrect')
bottom_bar = mpatches.Patch(color='lightblue', label='correct')
plt.legend(handles=[top_bar, bottom_bar])


In [ ]:
len(dat.loc[dat.group == 'Autism'].prolific_id.unique())

In [ ]:
len(dat.loc[dat.group == 'Autism'].prolific_id.unique())


In [ ]:
sns.catplot(col='belief_type', x='correct', hue='ascription_type', kind='count', data=dat)


## Check to make sure all responses came AFTER the target onset

In [9]:
sum(dat.apply(lambda row: row['response_onset'] > row['target_onset'], axis=1)) == len(dat)


True

## Exclude subjects with mean RTs less than 1500ms or more than 4000ms

In [ ]:
xx = dat.groupby('prolific_id').rt_ms.mean() < 1500
dat = dat.loc[dat.prolific_id.isin(xx[xx == False].index.tolist())]
yy = dat.groupby('prolific_id').rt_ms.mean() > 4000
dat = dat.loc[dat.prolific_id.isin(yy[yy == False].index.tolist())]

for old, new in {'IG': 'Ignorance', 'TB': 'True Info', 'FB': 'False Info'}.items():
    dat.belief_type.replace(old, new, inplace=True)

# 3. Location

In [32]:
adat = dat.loc[dat.group == 'Autism']
adat[['city', 'region', 'country', 'loc', 'org', 'postal', 'timezone']] = None

for subj in adat.prolific_id.unique():
    addr = adat.loc[adat.prolific_id == subj, 'ip_addy'].values[0]
    url = 'https://ipinfo.io/' + addr + '/json'
    res = urlopen(url)
    #response from url(if res==None then check connection)
    data = load(res)
    adat.loc[adat.prolific_id == subj, ['city', 'region', 'country', 'loc', 'org', 'postal', 'timezone', 'email']] = [data['city'], data['region'], data['country'], data['loc'], data['org'], data['postal'], data['timezone'], adat.loc[adat.prolific_id == subj, 'email'].values[0]]

<ipython-input-32-9e316d60e405>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adat[['city', 'region', 'country', 'loc', 'org', 'postal', 'timezone']] = None
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to 

In [35]:
xx = adat.groupby(['prolific_id','email', 'ip_addy','city', 'region', 'country', 'loc', 'org', 'postal', 'timezone']).correct.mean()#.sort_values('correct')
yy = xx.reset_index().sort_values('correct', axis=0)
yy

,prolific_id,email,ip_addy,city,region,country,loc,org,postal,timezone,correct
78,jwQ9QeuQD5HgcPDh,damoogu@outlook.com,107.150.23.229,Atlanta,Georgia,US,"33.7525,-84.3888",AS8100 QuadraNet Enterprises LLC,30303,America/New_York,0.000000
54,YBHK0TgAZz3oUD47,sakirudeenk020@gmail.com,92.119.177.91,New York City,New York,US,"40.7143,-74.0060",AS9009 M247 Ltd,10004,America/New_York,0.000000
55,YKz7pyUJbPXRbW60,tokulapeter13@gmail.com,199.66.147.3,El Campo,Texas,US,"29.1966,-96.2697","AS13943 Y K Communications, LTD",77437,America/Chicago,0.000000
56,YLxXu7mLl2wNkw1z,frankiedoyle437@gmail.com,86.107.55.57,New York City,New York,US,"40.7143,-74.0060",AS9009 M247 Ltd,10004,America/New_York,0.000000
83,mZB6bnc6D9y36FgC,ugochukwu.mbamalu.192718@unn.edu.ng,86.107.55.85,New York City,New York,US,"40.7143,-74.0060",AS9009 M247 Ltd,10004,America/New_York,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
46,Tpmax7tb29V3pRAf,alex@alexstave.com,73.242.70.58,Saint Paul,Minnesota,US,"44.9372,-93.1209","AS7922 Comcast Cable Communications, LLC",55102,America/Chicago,0.666667
84,nw2hzC6gbfiyXUQ6,itmake636@gmail.com,191.101.61.231,Las Vegas,Nevada,US,"36.1725,-115.1414",AS174 Cogent Communications,89162,America/Los_Angeles,0.750000
97,tnGByjBjbUzRJwyf,dave@trahans.net,191.96.227.102,New York City,New York,US,"40.7143,-74.0060",AS174 Cogent Communications,10004,America/New_York,0.833333
89,qU9fuMYwWHtEvYSY,judegarner0@gmail.com,62.128.207.107,Glasgow,Scotland,GB,"55.8651,-4.2576",AS20860 IOMART CLOUD SERVICES LIMITED,G1,Europe/London,0.833333


# 2. Response Times

In [ ]:
sns.set_context('notebook', font_scale=2)
fig, ax = plt.subplots(figsize=(15, 12))
g = sns.violinplot(x='belief_type', y='rt_ms', hue='ascription_type', hue_order=['Thinks', 'Knows'], data=dat.loc[(dat.trial_type=='test') & (dat.correct==True)], cut=0, bw=0.25, ax=ax, inner=None, linewidth=1, saturation=1)
g = sns.stripplot(x='belief_type', y='rt_ms', hue='ascription_type', hue_order=['Thinks', 'Knows'], data=dat.loc[(dat.trial_type=='test') & (dat.correct==True)],
                  color='k', dodge=True, ax=ax, jitter=.2, linewidth=1, alpha=.2)
g= sns.pointplot(x='belief_type', y='rt_ms', hue='ascription_type', hue_order=['Thinks', 'Knows'], data=dat.loc[(dat.trial_type=='test') & (dat.correct==True)], join=False, dodge=.4, capsize=.1, ax=ax, color='k',)

ax.legend(g.get_legend_handles_labels()[0][:2], g.get_legend_handles_labels()[1][:2])
g.yaxis.set_label_text("response time (ms)")
g.xaxis.set_label_text("Agent state")
ax.xaxis.labelpad = 20

#fig.savefig('proposal_S1_control_result.png')

In [ ]:
len(dat.loc[(dat.trial_type=='test') & (dat.correct==True)].prolific_id.unique())

# 3. Felicity Judgements

In [ ]:
f_dat = pd.read_sql('SELECT * FROM felicities A INNER JOIN subjects B ON A.prolific_id = B.prolific_id WHERE B.completion_code="XXXX"', con=conn)
f_dat = f_dat.loc[:,~f_dat.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate
# 1= "Sounds very weird"
# 7 = "Sounds very normal"
f_dat['Infelicity Rating'] = 8 - f_dat.felicity_rating

In [ ]:
sns.set_context('paper', font_scale=2)
fig, ax = plt.subplots(figsize=(12,12))
g = sns.violinplot(x='fel_belief_type', y='Infelicity Rating', hue='fel_ascription_type', data=f_dat, cut=0, bw=0.25, ax=ax, inner=None, linewidth=1, saturation=1, order=['TB', 'IG', 'FB'])

def jitter(values,j):
    return values + np.random.normal(j,0.15,values.shape)

g = sns.stripplot(x='fel_belief_type', y=jitter(f_dat['Infelicity Rating'], 0), hue='fel_ascription_type', data=f_dat,
                  color='k', dodge=True, ax=ax, jitter=.45, linewidth=1.5, alpha=.1, order=['TB', 'IG', 'FB'])
g= sns.pointplot(x='fel_belief_type', y='Infelicity Rating', hue='fel_ascription_type', data=f_dat, join=False, dodge=.4, capsize=.1, ax=ax, color='k',order=['TB', 'IG', 'FB'])

ax.legend(g.get_legend_handles_labels()[0][:2], g.get_legend_handles_labels()[1][:2])

# 4. AQ-10

In [ ]:

a_dat = pd.read_sql('SELECT * FROM autism_scores A INNER JOIN subjects B ON A.prolific_id = B.prolific_id WHERE B.completion_code="XXXX"', con=conn)
a_dat = a_dat.loc[:,~a_dat.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate

#a_dat = a_dat[a_dat.prolific_id.isin(dat.prolific_id.unique()).values] # only use subjects in trial analysis
a_dat.describe()

In [ ]:
a_dat[a_dat.prolific_id.isin(dat.prolific_id.unique().tolist())]

In [ ]:
## Scoring
agree = ['AQ_rating_1', 'AQ_rating_7', 'AQ_rating_8', 'AQ_rating_10'] # score of 3 or 4 get a point
disagree = ['AQ_rating_2','AQ_rating_3','AQ_rating_4','AQ_rating_5','AQ_rating_6','AQ_rating_9', ] # score of 1 or 2 get a point]
a = a_dat[agree] >= 3
b = a_dat[disagree] < 3
a_dat['AQ_score'] = a.sum(axis=1) + b.sum(axis=1)
a_dat = a_dat[a_dat.prolific_id.isin(dat.prolific_id.unique().tolist())]

fg, ax = plt.subplots(figsize=(10, 8))
sns.countplot(data=a_dat, x='AQ_score', ax=ax);

In [ ]:
e_dat.groupby('diag').describe()

In [ ]:
cdat = dat.loc[dat.correct ==True]
for subject in dat.prolific_id.unique():
    # calculate mean RT for belief ascription
    a_dat.loc[a_dat.prolific_id == subject, 'avg_ThinkRT'] = cdat.loc[(cdat.prolific_id == subject) & (cdat.ascription_type == 'Thinks')].rt_ms.mean()
    a_dat.loc[a_dat.prolific_id == subject, 'avg_KnowRT'] = cdat.loc[(cdat.prolific_id == subject) & (cdat.ascription_type == 'Knows')].rt_ms.mean()

a_dat['RT_diff'] = a_dat.apply(lambda row: row['avg_ThinkRT'] - row['avg_KnowRT'], axis=1)


In [ ]:
sns.lmplot(x='AQ_score', y='RT_diff', data=a_dat);
